# Forecasting & Inventory Alert System

An end-to-end ingredient demand forecasting & inventory alert system for a café, designed from raw transactional data to procurement signals

Creator: [Andrew Virya Victorio](https://avv.framer.website/) <br>
Email: [andrew.avv03@gmail.com](mailto:andrew.avv03@gmail.com) <br>
Repository: [github.com/AlphaByte-RedTeam/forecasting-inventory-alert-system](https://github.com/AlphaByte-RedTeam/forecasting-inventory-alert-system) <br>

Created date: Dec 17th, 2025

NB: This data is using a pre-processed and normalized datasets from local cafe nearby that cannot be disclosed. Thank you for understanding!

## Column Explanation

### 1. Sales Data
Filename: coffee_shop_sales_detailed_2y.csv <br>
Grain: 1 row = 1 drink sold

| Column           | Explanation                                                |
| ---------------- | ---------------------------------------------------------- |
| `date`           | Date when the drink was sold                               |
| `product`        | Base beverage ordered                                      |
| `temperature`    | Serving type: hot or ice                                   |
| `sugar_level`    | Customer sugar preference (less / normal / extra)          |
| `ice_level`      | Ice quantity selected (none / less / normal)               |
| `coffee_used`    | Quantity of coffee beans used                              |
| `coffee_unit`    | Measurement unit for coffee (g)                            |
| `milk_used`      | Quantity of milk used                                      |
| `milk_unit`      | Measurement unit for milk (ml)                             |
| `chocolate_used` | Quantity of chocolate powder used                          |
| `chocolate_unit` | Measurement unit for chocolate (g)                         |
| `sugar_used`     | Quantity of sugar used                                     |
| `sugar_unit`     | Measurement unit for sugar (g)                             |
| `water_used`     | Quantity of water used                                     |
| `water_unit`     | Measurement unit for water (ml)                            |
| `is_weekend`     | Indicates if the sale occurred on a weekend                |
| `is_promo`       | Indicates if the sale occurred on a promo date (1.1–12.12) |

### 2. Ingredients Data
Filename: coffee_shop_inventory_2y.csv <br>
Grain: 1 row = 1 ingredient per day

| Column            | Explanation                                     |
| ----------------- | ----------------------------------------------- |
| `date`            | Inventory snapshot date                         |
| `ingredient_name` | Ingredient identifier                           |
| `ingredient_unit` | Measurement unit of the ingredient              |
| `opening_stock`   | Stock level at the start of the day             |
| `used_today`      | Total ingredient usage for the day              |
| `closing_stock`   | Remaining stock after usage                     |
| `reorder_point`   | Stock threshold to trigger replenishment        |
| `lead_time_days`  | Supplier lead time in days                      |
| `reorder_flag`    | Indicates whether a reorder should be triggered |

Note: All ingredient quantities are stored with explicit unit columns to ensure safe aggregation, forecasting, and long-term schema extensibility.

## Load & read the dataset

In [1]:
import pandas as pd
import os

current_dir = os.getcwd()
dataset_dir = f'{current_dir}/datasets'

df_inventory = pd.read_csv(f'{dataset_dir}/coffee_shop_inventory_2y.csv')
df_sales = pd.read_csv(f'{dataset_dir}/coffee_shop_sales_detailed_2y.csv')

In [2]:
df_inventory.head()

,date,ingredient_name,ingredient_unit,opening_stock,used_today,closing_stock,reorder_point,lead_time_days,reorder_flag
0,2023-01-01,coffee,g,50000.0,5004.0,44996.0,15000,7,False
1,2023-01-01,milk,ml,200000.0,29420.0,170580.0,60000,3,False
2,2023-01-01,chocolate,g,30000.0,850.0,29150.0,8000,5,False
3,2023-01-01,sugar,g,40000.0,3094.5,36905.5,10000,4,False
4,2023-01-02,coffee,g,44996.0,2610.0,42386.0,15000,7,False


In [3]:
df_sales.head()

,date,product,temperature,sugar_level,ice_level,coffee_used,coffee_unit,milk_used,milk_unit,chocolate_used,chocolate_unit,sugar_used,sugar_unit,water_used,water_unit,is_weekend,is_promo
0,2023-01-01,Espresso,hot,extra,none,18,g,0,ml,0,g,12.0,g,30.0,ml,True,True
1,2023-01-01,Espresso,hot,normal,none,18,g,0,ml,0,g,10.0,g,30.0,ml,True,True
2,2023-01-01,Espresso,hot,normal,none,18,g,0,ml,0,g,10.0,g,30.0,ml,True,True
3,2023-01-01,Espresso,ice,normal,normal,18,g,0,ml,0,g,10.0,g,30.0,ml,True,True
4,2023-01-01,Espresso,hot,less,none,18,g,0,ml,0,g,7.5,g,30.0,ml,True,True


## Data Quality & Validation

In [4]:
# Check datafram data types
df_inventory.dtypes

date                object
ingredient_name     object
ingredient_unit     object
opening_stock      float64
used_today         float64
closing_stock      float64
reorder_point        int64
lead_time_days       int64
reorder_flag          bool
dtype: object

In [5]:
df_sales.dtypes

date               object
product            object
temperature        object
sugar_level        object
ice_level          object
coffee_used         int64
coffee_unit        object
milk_used           int64
milk_unit          object
chocolate_used      int64
chocolate_unit     object
sugar_used        float64
sugar_unit         object
water_used        float64
water_unit         object
is_weekend           bool
is_promo             bool
dtype: object

In [6]:
# Check missing or null values from the dataset

inventory_na = df_inventory.columns[df_inventory.isna().any()].tolist()
inventory_null = df_inventory.columns[df_inventory.isnull().any()].tolist()

sales_na = df_sales.columns[df_sales.isna().any()].tolist()
sales_null = df_sales.columns[df_sales.isnull().any()].tolist()

print(f'Inventory NA: {inventory_na}')
print(f'Inventory NULL: {inventory_null}')
print(f'Sales NA: {sales_na}')
print(f'Sales NULL: {sales_null}')

Inventory NA: []
Inventory NULL: []
Sales NA: []
Sales NULL: []


In [7]:
# Check negative values
inventory_negative = df_inventory[(df_inventory.select_dtypes(include=['number']) < 0).any(axis=1)]
sales_negative = df_sales[(df_sales.select_dtypes(include=['number']) < 0).any(axis=1)]

print(f'Inventory Negative Values:\n{inventory_negative}')
print(f'Sales Negative Values:\n{sales_negative}')

Inventory Negative Values:
            date ingredient_name ingredient_unit  opening_stock  used_today  \
41    2023-01-11            milk              ml        13940.0     16020.0   
45    2023-01-12            milk              ml        -2080.0     14480.0   
49    2023-01-13            milk              ml       -16560.0     21500.0   
53    2023-01-14            milk              ml       -38060.0     20440.0   
57    2023-01-15            milk              ml       -58500.0     24340.0   
...          ...             ...             ...            ...         ...   
2919  2024-12-30           sugar               g     -1307199.5      1583.0   
2920  2024-12-31          coffee               g     -2106166.0      2574.0   
2921  2024-12-31            milk              ml    -13216980.0     14860.0   
2922  2024-12-31       chocolate               g      -379600.0       425.0   
2923  2024-12-31           sugar               g     -1308782.5      1583.5   

      closing_stock  reo

In [8]:
# Check all ingredient_name available
ingredients = set(df_inventory['ingredient_name'].unique().tolist())

# Grouping per date per ingredient name
daily_ingredients = df_inventory.groupby('date')['ingredient_name'].apply(set)

# Find dates with missing required ingredients
mismatched_dates = daily_ingredients[daily_ingredients != ingredients]
print(mismatched_dates)

Series([], Name: ingredient_name, dtype: object)


## Data Aggregation

In [9]:
# daily ingredient consumption
daily_consumption = df_inventory.groupby(['date', 'ingredient_name'])['used_today'].sum().unstack(fill_value=0)
daily_consumption.index = pd.to_datetime(daily_consumption.index)

daily_consumption.head()

ingredient_name,chocolate,coffee,milk,sugar
date,,,,
2023-01-01,850.0,5004.0,29420.0,3094.5
2023-01-02,375.0,2610.0,15060.0,1562.0
2023-01-03,475.0,2340.0,15440.0,1486.0
2023-01-04,550.0,2826.0,18620.0,1749.5
2023-01-05,525.0,2574.0,16800.0,1617.0


In [ ]:
# sanity check
daily_consumption.loc["2023-01-02":"2023-01-08"].sum()

ingredient_name
chocolate      3650.0
coffee        20304.0
milk         124840.0
sugar         12660.5
dtype: float64

In [12]:
# weekly ingredient consumption
weekly_consumption = daily_consumption.resample('W-SUN').sum()
weekly_consumption.head()

ingredient_name,chocolate,coffee,milk,sugar
date,,,,
2023-01-01,850.0,5004.0,29420.0,3094.5
2023-01-08,3650.0,20304.0,124840.0,12660.5
2023-01-15,3750.0,20880.0,128580.0,12936.0
2023-01-22,3700.0,20322.0,127640.0,12627.0
2023-01-29,3575.0,19602.0,120280.0,12233.0


In [15]:
weekly_consumption.loc["2023-01-08"]

ingredient_name
chocolate      3650.0
coffee        20304.0
milk         124840.0
sugar         12660.5
Name: 2023-01-08 00:00:00, dtype: float64

In [16]:
# weekly ingredient consumption
monthly_consumption = weekly_consumption.resample('ME').sum()
monthly_consumption.head()

ingredient_name,chocolate,coffee,milk,sugar
date,,,,
2023-01-31,15525.0,86112.0,530760.0,53551.0
2023-02-28,15550.0,81648.0,516200.0,50986.0
2023-03-31,15850.0,83484.0,517100.0,52132.0
2023-04-30,19300.0,101340.0,629240.0,63611.0
2023-05-31,15125.0,83502.0,518560.0,51954.5
